# `apps_long.csv`

In [77]:
library(dplyr)
library(readr)
library(tidyr)

cr_apps <- read_csv("../output/apps_long.csv", show_col_types = FALSE)

pl_ssns <- read_csv("../output/player_ssns.csv", show_col_types = FALSE)

gls <- read_csv("../output/scorers-long.csv", show_col_types = FALSE) %>%
    group_by(
        season,
        game_no,
        player_name
    ) %>%
    summarise(
        goals_scored = sum(goals_scored),
        .groups = "drop"
    )

res <- read_csv("https://raw.githubusercontent.com/petebrown/update-results/main/data/results_df.csv", show_col_types = FALSE)

In [78]:
library(stringr)

sb_game_ids <- read_csv("https://raw.githubusercontent.com/petebrown/update-player-stats/main/data/players_df.csv", show_col_types = FALSE) %>%
    select(
        sb_game_id,
        game_date        
    ) %>%
    rename(
        game_id = sb_game_id
    ) %>%
    mutate(
        game_id = str_remove(game_id, "tpg"),
        game_id = as.numeric(game_id)
    ) %>%
    unique() %>%
    arrange(
        game_date
    )

head(sb_game_ids, 3)

game_id,game_date
<dbl>,<date>
240309,1996-08-17
243200,1996-08-21
242383,1996-08-23


#### Pivot apps_long to wide format mimicking book

In [79]:
apps_long <- cr_apps %>%
    left_join(
        pl_ssns %>%
            select(
                season,
                disam_name,
                player_name
            ) %>% unique(),
        by = c("season" = "season", "player_name" = "disam_name")
    ) %>%
    mutate(
        player_name = player_name.y
    ) %>%
    select(-player_name.y) %>%
    left_join(
        gls,
        by = c("season" = "season", "game_no" = "game_no", "player_name" = "player_name")
    ) %>%
    left_join(
        res %>%
            select(
                season,
                ssn_game_no,
                generic_comp,
                competition
            ),
        by = c("season" = "season", "game_no" = "ssn_game_no")
    ) %>%
    mutate(
        generic_comp = ifelse(str_detect(competition, "lay"), "League Play-Off", generic_comp)
    ) %>%
    group_by(
        season,
        player_name,
        generic_comp,
        role
    ) %>%
    summarise(
        count = n(),
        .groups = "drop"
    )

# Print example for one season and player
apps_long %>%
    filter(
        season == "1992/93",
        player_name == "Johnny Morrissey"
    )    

season,player_name,generic_comp,role,count
<chr>,<chr>,<chr>,<chr>,<int>
1992/93,Johnny Morrissey,Anglo-Italian Cup,starter,6
1992/93,Johnny Morrissey,FA Cup,starter,3
1992/93,Johnny Morrissey,Football League,starter,42
1992/93,Johnny Morrissey,Football League,sub,1
1992/93,Johnny Morrissey,League Cup,starter,1
1992/93,Johnny Morrissey,League Cup,sub,1
1992/93,Johnny Morrissey,League Play-Off,starter,2


In [80]:
gls_long <- cr_apps %>%
    left_join(
        pl_ssns %>%
            select(
                season,
                disam_name,
                player_name
            ) %>% unique(),
        by = c("season" = "season", "player_name" = "disam_name")
    ) %>%
    mutate(
        player_name = player_name.y
    ) %>%
    select(-player_name.y) %>%
    left_join(
        gls %>%
            select(
                season,
                game_no,
                player_name,
                goals_scored
            ),
        by = c("season", "game_no", "player_name")
    ) %>%
    select(
        season,
        game_no,
        player_name,,
        goals_scored
    ) %>%
    left_join(
        res %>%
            select(
                season,
                ssn_game_no,
                generic_comp,
                competition
            ),
        by = c("season" = "season", "game_no" = "ssn_game_no")
    ) %>%
    mutate(
        goals_scored = ifelse(is.na(goals_scored), 0, goals_scored),
        generic_comp = ifelse(str_detect(competition, "lay"), "League Play-Off", generic_comp)
    ) %>%
    group_by(
        season,
        player_name,
        generic_comp
    ) %>%
    summarise(
        count = sum(goals_scored),
        .groups = "drop"
    ) %>%
    mutate(
        role = "goals"
    )

# Print example for one season and player
gls_long %>%
    filter(
        season == "1992/93",
        player_name == "John Aldridge"
    )

season,player_name,generic_comp,count,role
<chr>,<chr>,<chr>,<dbl>,<chr>
1992/93,John Aldridge,Anglo-Italian Cup,2,goals
1992/93,John Aldridge,FA Cup,1,goals
1992/93,John Aldridge,Football League,21,goals
1992/93,John Aldridge,League Cup,2,goals
1992/93,John Aldridge,League Play-Off,0,goals


In [81]:
# Change options so all columns show for season
options(repr.matrix.max.cols=50, repr.matrix.max.rows=100)

In [82]:
rbind(apps_long, gls_long) %>%
    mutate(
        generic_comp = ifelse(generic_comp %in% c("Football League", "Non-League"), "League", generic_comp)
    ) %>%
    filter(
        season == "1992/93"
    ) %>%
    mutate(
        generic_comp = factor(
            generic_comp,
            levels = c("League", "FA Cup", "League Cup", "Associate Members' Cup", "Anglo-Italian Cup", "Full Members' Cup", "FA Trophy", "League Play-Off")
        ),
        role = factor(
            role,
            levels = c("starter", "sub", "goals")
        )
    ) %>%
    left_join(
        pl_ssns %>% select(season, player_name, disam_name) %>% unique(),
        by = c("season" = "season", "player_name" = "player_name")
    ) %>%
    arrange(
        season,
        disam_name
    ) %>%
    pivot_wider(
        id_cols = c(season, generic_comp, role),
        names_from = disam_name,
        values_from = count,
        values_fill = 0
    ) %>%
    arrange(
        season,
        generic_comp,
        role
    )

season,generic_comp,role,Aldridge,Branch,Brannan,Cooper,Coyne D,Coyne T,Garnett,Higgins,Hughes,Irons,Malkin,Martindale,McGreal,McNab,Morrissey,Muir,Mungall,Nevin,Nixon,Nolan,Proctor,Thomas,Vickers
<chr>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1992/93,League,starter,29,0,38,3,1,9,5,40,7,40,35,18,0,30,42,7,29,43,45,14,13,16,42
1992/93,League,sub,1,3,0,3,0,3,0,0,4,2,1,8,0,1,1,4,6,0,0,0,0,0,0
1992/93,League,goals,21,0,1,0,0,1,1,4,0,7,7,2,0,2,5,2,3,13,0,0,1,0,0
1992/93,FA Cup,starter,2,0,1,0,0,0,0,2,1,3,3,0,0,3,3,1,3,3,3,2,0,0,3
1992/93,FA Cup,sub,0,0,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0
1992/93,FA Cup,goals,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,1
1992/93,League Cup,starter,2,0,2,0,0,0,2,0,2,2,2,1,0,1,1,0,0,1,2,2,0,2,0
1992/93,League Cup,sub,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0
1992/93,League Cup,goals,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
